Liberaries

In [3]:

from re import sub
from skimage.io import  imread, imshow
from skimage.transform import  resize, rescale
from skimage.color import rgb2gray
import numpy as np
from os import listdir, path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os,os.path
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

Data processing 

In [4]:

SEED = 42
IMG_SIZE = 224  # ResNet50 input size
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

np.random.seed(SEED)
tf.random.set_seed(SEED)
import os
data='/kaggle/input/plantvillage-dataset/color'

def create_dataframe(data_path):
    filepaths = []
    labels = []
    folds = os.listdir(data_path)

    for fold in folds:
        f_path = os.path.join(data_path, fold)
        imgs = os.listdir(f_path)

        for img in imgs:
            img_path = os.path.join(f_path, img)

            filepaths.append(img_path)
            labels.append(fold)

    fseries = pd.Series(filepaths, name='Filepaths')
    lseries = pd.Series(labels, name='Labels')

    return pd.concat([fseries, lseries], axis=1)                                         
df = create_dataframe(data)

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
train_df, dummy_df = train_test_split(df,  train_size= 0.8, shuffle= True, random_state= 42)
valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 42)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255) 
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepaths',
    y_col='Labels',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='Filepaths',
    y_col='Labels',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepaths',
    y_col='Labels',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['Labels'])

num_classes = len(label_encoder.classes_)
print(f"\nNumber of classes: {num_classes}")
print("\nChecking class distribution...")

class_counts = df['Labels'].value_counts()
print(f"Classes with < 3 samples: {sum(class_counts < 3)}")

valid_classes = class_counts[class_counts >= 3].index
df_filtered = df[df['Labels'].isin(valid_classes)].copy()

print(f"Original dataset: {len(df)} images, {len(df['Labels'].unique())} classes")
print(f"Filtered dataset: {len(df_filtered)} images, {len(df_filtered['Labels'].unique())} classes")

label_encoder = LabelEncoder()
df_filtered['label_encoded'] = label_encoder.fit_transform(df_filtered['Labels'])
num_classes = len(label_encoder.classes_)   # <-- FIXED

X = df_filtered['Filepaths'].values
y = df_filtered['label_encoded'].values


X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=SEED, stratify=y_train_val
)

print(f"\nDataset split:")
print(f"Train: {len(X_train)} images")
print(f"Validation: {len(X_val)} images")
print(f"Test: {len(X_test)} images")

Found 43444 validated image filenames belonging to 38 classes.
Found 5430 validated image filenames belonging to 38 classes.
Found 5431 validated image filenames belonging to 38 classes.

Number of classes: 38

Checking class distribution...
Classes with < 3 samples: 0
Original dataset: 54305 images, 38 classes
Filtered dataset: 54305 images, 38 classes

Dataset split:
Train: 34755 images
Validation: 8689 images
Test: 10861 images
